# Task 1

In [ ]:
import os
import cv2
import numpy as np

CUR_DIR = os.getcwd()

# Reading the image file, in the BGR color space.
image = cv2.imread(os.path.join(CUR_DIR, "for_watson.png"), cv2.IMREAD_COLOR)

# Transforming BGR image to Grayscale - MESSAGE : My Dear Watson,
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow("image_gray", image_gray)
cv2.imwrite("for_watson_gray.png", image_gray)

# Sharpening the grayscale image using a kernel of size 3x3.
kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
image_gray_sharp = cv2.filter2D(image_gray, -1, kernel)
cv2.imshow("image_gray_sharp", image_gray_sharp)
cv2.imwrite("for_watson_gray_sharp.png", image_gray_sharp)

# Press 0 to destroy all windows.
cv2.waitKey(0)
cv2.destroyAllWindows()


# Task 2

# Task 3

## Calibrate camera using photos of AprilTag markers

In [ ]:
"""
Calibrate camera using AprilTag markers
"""

import importlib
import os
import sys

import pyAprilTag

CUR_DIR = os.getcwd()
LOG_DIR = os.path.join(CUR_DIR, "calib_log")
sys.path.insert(0, CUR_DIR)  # otherwise importlib cannot find the path

# If calib_log doesn't exist, create it.
if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)
    pyAprilTag.calib(
        pyAprilTag.calib_pattern_path,
        "photo://{}".format(os.path.join(pyAprilTag.calib_example_dir, "*.png")),
        log_dir=LOG_DIR,
        nDistCoeffs=4,
    )

logs = sorted([f for f in os.listdir(LOG_DIR) if f.endswith(".py")])
if len(logs) == 0:
    print("no calibration log available!")
    exit(-1)

last_log = os.path.relpath(os.path.join(LOG_DIR, logs[-1])).replace(os.path.sep, ".")[
    :-3
]

calib = importlib.import_module(last_log)
print("last log: " + last_log)
print("camera intrinsic matrix:")
print(calib.K)
print("camera distortion parameters:")
print(calib.distCoeffs)


## Calibrate camera using video feed from the camera

In [ ]:
"""
Calibrate camera using AprilTag markers
"""

import importlib
import os
import sys

import pyAprilTag

CUR_DIR = os.getcwd()
LOG_DIR = os.path.join(CUR_DIR, "calib_log")
sys.path.insert(0, CUR_DIR)  # otherwise importlib cannot find the path
if (
    not os.path.exists(LOG_DIR)
    or len([f.endswith(".png") for f in os.listdir(LOG_DIR)]) < 2
):
    os.makedirs(LOG_DIR)
    pyAprilTag.calib(
        pyAprilTag.calib_pattern_path,
        "camera://{:d}".format(0),
        log_dir=LOG_DIR,
        nDistCoeffs=4,
    )

logs = sorted([f for f in os.listdir(LOG_DIR) if f.endswith(".py")])
if len(logs) == 0:
    print("no calibration log available!")
    exit(-1)

last_log = os.path.relpath(os.path.join(LOG_DIR, logs[-1])).replace(os.path.sep, ".")[
    :-3
]
calib = importlib.import_module(last_log)

print(f"last log: {last_log}")
print(f"camera intrinsic matrix: {calib.K}")
print(f"camera distortion parameters: {calib.distCoeffs}")


# Task 4

In [ ]:
import importlib
import os
import sys

import cv2
import numpy as np
import pyAprilTag


def compute_camera_matrix(K: np.array, R: np.array, t: np.array) -> np.array:
    return K @ np.r_[np.c_[R, t]]  # 3x4 array


def draw_cube(K, image):
    ids, corners, centers, Hs = pyAprilTag.find(image)
    num, R, t, N = cv2.decomposeHomographyMat(Hs, K)
    print(num, R, t, N)


def get_calib_from_log():
    CUR_DIR = os.getcwd()
    LOG_DIR = os.path.join(CUR_DIR, "calib_log")
    sys.path.insert(0, CUR_DIR)  # otherwise importlib cannot find the path

    logs = sorted([f for f in os.listdir(LOG_DIR) if f.endswith(".py")])
    if len(logs) == 0:
        print("no calibration log available!")
        exit(-1)

    last_log = os.path.relpath(os.path.join(LOG_DIR, logs[-1])).replace(
        os.path.sep, "."
    )[:-3]
    return importlib.import_module(last_log)


def main():

    # get calibration matrix
    K = get_calib_from_log().K
    distCoeffs = get_calib_from_log().distCoeffs
    
    capture = cv2.VideoCapture(0)
    # read the images
    image1 = cv2.imread("image1.jpg")
    image2 = cv2.imread("image2.jpg")

    # draw the cube
    draw_cube(K, image1)
    draw_cube(K, image2)

    # find apriltags in the images
    ids, corners, centers, Hs = pyAprilTag.find(image1)
    ids, corners, centers, Hs = pyAprilTag.find(image2)
    
    # cv2.imshow("image2", image2)
    # cv2.waitKey(0)


if __name__ == "__main__":
    main()
